In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

In [3]:
# soup = BeautifulSoup(r.content, 'lxml') #usamos lxml como parser

¿Qué queremos obtener de la página?
1. Nombre del producto
2. Precio (?)
3. Detalles
4. Ingredientes

## 1 OBTENER TODAS LAS URLS DE LOS PRODUCTOS

Para ello, utilizaremos bucles que iteren sobre los elementos de la página.
* Variable global cleansers_url, es el contenedor de todas las urls.
* Los links están dentro de un tag 'a', como sólo nos interesan aquellas que tengan href, pues href=True. 
* Se ha agregado class='product-image" porque si no, el bucle cogía todas las urls (wishlist, url de la imagen etc), y solo nos interesan los productos.
* Append cada uno de los links al contenedor.

cream_list = soup.find_all('div', class_='itemContainer')

url = []
for item in cream_list:
        for link in item.find_all('a', href=True): 
            url.append(link['href'])

In [6]:
scrubs_url = []
for i in range (1, 13):
    url = f'https://www.yesstyle.com/en/women-facial-scrubs/list.html/bcc.14336_bpt.46#/bt=37&sb=136&bpt=46&pn={i}&s=10&bcc=14336&l=1' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    scrubs_list = soup.find_all('div', class_='itemContainer')
    for item in scrubs_list:
        for link in item.find_all('a', href=True): 
            scrubs_url.append(link['href'])


In [7]:
print(len(scrubs_url))

432


with open("cleansers_url.txt", "w") as output:
    output.write(str(cleansers_url))

## 2 OBTENER LA INFO DE LA PÁGINA PROPIA DEL PRODUCTO
Info que necesitamos:
* Nombre del producto
* Details
* Ingredients

* Details e Ingredients se encuentran dentro de un grupo <div class='panel-group'

In [51]:
"""
test_url = 'https://www.yesstyle.com/es/cosrx-gel-limpiador-con-ph-bajo-good-morning-150ml/info.html/pid.1052684732'
r = requests.get(test_url) #para cada link en el listado, hacemos una requests
soup = BeautifulSoup(r.content, 'lxml')
name = soup.find_all('div', class_='headingContent flex-gt-sm-66 flex-100')
for x in name:
    title = soup.find('h1').text
ingred = soup.find('div', class_="cropWrapper").text #ingredients
rating = soup.find('div', class_='satisfied').text
"""

In [6]:
scrubs_url[0]

'https://www.yesstyle.com/en/the-face-shop-real-nature-face-mask-1pc-20-types-20g-calendula/info.html/pid.1061020764'

In [8]:
scrubs = []

In [9]:
for link in scrubs_url[0:432]:
    r = requests.get(link, headers=headers) #para cada link en el listado, hacemos una requests
    soup = BeautifulSoup(r.content, 'lxml')
    #usamos try, except porque hay ocasiones en las que no hay info sobre los ingredientes
    try:
        name_box = soup.find_all('div', class_='headingContent flex-gt-sm-66 flex-100')
        for x in name_box:
            name = soup.find('h1').text
        ingredients = soup.find('div', class_="cropWrapper").text.strip() #ingredients
        rating = soup.find('div', class_='satisfied').text.strip()
    except:
        name= 'no info'
        ingredients = 'no info'
        rating = 'no info'
        
    scrubs_info = {
    'name': name,
    'rating': rating,
    'ingredients': ingredients}
    
    scrubs.append(scrubs_info)

In [10]:
print(len(scrubs))

432


In [12]:
scrubs[496]

IndexError: list index out of range

## 3. PASARLO A DATAFRAME

In [11]:
scrubs_df = pd.DataFrame(scrubs)
scrubs_df

,name,rating,ingredients
0,\nSecret Key - Lemon Sparkling Peeling Gel 120...,"97.9% customers satisfied|3,356 reviews","Purified Water, Cetrimonium Chloride, Cellulos..."
1,\nHOLIKA HOLIKA - Pig Clear Black Head 3-Step ...,"90.3% customers satisfied|1,163 reviews","Hamamelis Virginiana (Witch Hazel) Water, Buty..."
2,\nPyunkang Yul - Low pH Pore Deep Cleansing Fo...,97.8% customers satisfied|248 reviews,"Water, Sodium Cocoyl Isethionate, Glycerin, Co..."
3,\nRiRe - Style Black Head Brush Cleanser 20ml\n,"90.5% customers satisfied|2,491 reviews","Water, Myristic Acid, Butylene Glycol, Stearic..."
4,\nElizavecca - Witch Piggy Hell Pore Control H...,"95.5% customers satisfied|1,030 reviews","Sodium Hyaluronate, Niacinamide, 1,2-Hexanedio..."
...,...,...,...
427,no info,no info,no info
428,\nSKINFOOD - Egg White Perfect Pore Cleansing ...,97.2% customers satisfied|117 reviews,"Water, Myristic Acid, Glycerin, Potassium Hydr..."
429,\nCOSRX - One Step Green Hero Calming Pad\n,96.8% customers satisfied|169 reviews,"Butylene Glycol, Taraxacum Officinale (Dandeli..."
430,\nTHE FACE SHOP - fmgt Lip Scrub\n,88.6% customers satisfied|51 reviews,"Water, Glycerin, Cetearyl Alcohol, Macadamia T..."


In [12]:
scrubs = scrubs_df.to_csv('yesstyle_data/scrubs.csv')